# matchbox の構築

## 鍵交換

maint ユーザを作成して適当にパスワードを設定する
```
# useradd maint
# passwd
# echo "maint ALL=(ALL) NOPASSWD: ALL" >> /etc/sudoers
# chown maint:maint /home/maint
# su - maint
$ cp /etc/skel/.??* .
$ exit
```

ssh でログインして、以下の鍵を登録する

In [ ]:
cat ~/.ssh/id_rsa.pub

## Python をインストール

ArchLinux に Python が入っていなかったので、raw モジュールでインストールする

In [ ]:
ansible matchbox -b -u maint -m raw -a "pacman -Syu --noconfirm python"

## Ansible の接続確認

In [ ]:
ansible matchbox -b -u maint -m ping

## Ansible で設定投入

In [ ]:
ansible-playbook matchbox.yml

## Matchbox の起動確認

In [ ]:
ansible matchbox -u maint -a "systemctl status matchbox"

## クライアント認証鍵を取得

In [ ]:
mkdir -p ~/.matchbox
for file in ca.crt client.crt client.key; do
  ansible matchbox -b -u maint -a "cat ~/tls/$file" | sed 1d > ~/.matchbox/$file
  echo "OK $file"
done

## 鍵を削除

server.key, client.keyを再作成するとき以外は実行しないでください

In [ ]:
#ansible matchbox -b -u maint -m file -a "path=~/tls state=absent"
#ansible matchbox -b -u maint -m systemd -a "name=matchbox state=stopped enabled=no"

## http 接続確認

In [ ]:
ansible matchbox --list-hosts | sed 1d | while read host; do
    curl http://$host:8080
done

In [ ]:
cat /etc/hosts

## rpc 接続確認

In [ ]:
ansible matchbox --list-hosts | sed 1d | while read host; do
  openssl s_client -connect $host:8081 -CAfile ~/.matchbox/ca.crt -cert ~/.matchbox/client.crt -key ~/.matchbox/client.key
done

## CoreOS イメージ取得

通信量が多いので、実行は極力控えること

In [ ]:
ansible matchbox -b -u maint -m shell -a "./get-coreos stable 1576.4.0 . chdir=/var/lib/matchbox/assets"

## CoreOS 取得確認

In [ ]:
ansible matchbox --list-hosts | sed 1d | while read host; do
    curl http://$host:8080/assets/coreos/1576.4.0/
done